<a href="https://colab.research.google.com/github/2kunal6/cuda_lab/blob/master/Session1/solutions/Assignment_1_Kunal_Rout%2C_Marc_Goedecke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

- Set Jupyter server in your home account so that you can use it remotely (Optional due to Covid-19!)
- Build 2 layer network for Logistic Regression Classifier using pure python/numpy 
    - Input data x : 20 x 784 (20 is the batch size)
    - label y : 20 x 10
    - Weight Variable W with random_normal initialization
    - Bias variable b with zeros
    - Use softmax activation => softmax(x.W+b)
    - Use OneHot encoding and MSE loss! Not the best way for classification but just for practice 
    - Train the model on MNIST
    - PS: If you dont know how to start see this: http://iamtrask.github.io/2015/07/12/basic-python-network/
- Submit your ipython notebook file to farazi@ais.uni-bonn.de with subject: Assignment1

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

train_dataset= dsets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset= dsets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!






/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
import numpy as np

In [0]:
# train dataset

# input dataset
X = train_dataset.data.numpy()
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])
X = np.array(X, dtype=np.float32)/255.

# output data (labels)
y = train_dataset.targets.numpy()
y_vector = np.eye(10)[y]

In [0]:
# test dataset

X_test = test_dataset.data.numpy()
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
X_test = np.array(X_test, dtype=np.float32)/255.

# output data (labels)
y_test = test_dataset.targets.numpy()
y_vector_test = np.eye(10)[y_test]

In [5]:
# example test dataset

%matplotlib inline
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from IPython.display import clear_output
from IPython.display import clear_output   

def on_value_change(change):
    print(change['new'])

int_range = widgets.IntSlider(
    value=7,
    min=0,
    max=10000-1,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(int_range)

def on_value_change(change):
    clear_output()
    display(int_range)
    
    print("label: " + str(y_test[int(change['new'])]))
    img = X_test[int(change['new'])].reshape(28,28)
    plt.imshow(img,cmap='gray')
    plt.colorbar()

int_range.observe(on_value_change, names='value')

IntSlider(value=7, continuous_update=False, description='Test:', max=9999)

In [0]:
# softmax function
def softmax(z,deriv=False):
    z = z.reshape(-1,1)
    if(deriv==True):
        z = z.reshape(-1,1)
        return np.diagflat(z) - np.dot(z, z.T)
    z -= np.max(z)
    return (np.exp(z).T / np.sum(np.exp(z), axis=0)).T

In [0]:
def softmax_batch(z,deriv=False):
    if(deriv==True):
        derivative = np.zeros((z.shape[1],z.shape[1]))
        for row in range(z.shape[0]):
            derivative += softmax(z[row][:], deriv)
        derivative /= z.shape[0]
        return derivative
        
    batch_z = np.zeros(z.shape)
    for row in range(z.shape[0]):
        batch_z[row][:] = softmax(z[row][:]).T
    
    return batch_z

In [9]:
num_labels = 10

# default accuracy 0 %
accuracy = 0

# seed random numbers to make calculation
np.random.seed(1)

# initialize weights with random normal initialization
weights = np.random.random((X.shape[1], num_labels))

# initialize bias with zeros
bias = np.zeros(num_labels)

# the optimal values
opt_weights = weights
opt_bias = bias


batch_size = 20
num_batches = int(X.shape[0] / batch_size) # 3000

no_refresh_counter = 0

for epoche in range(200):
    #print("Epoch: " + str(epoche))
    permutation = np.random.permutation(X.shape[0])
    np.take(X,permutation,axis=0,out=X);
    np.take(y_vector,permutation,axis=0,out=y_vector);
    
    for batch in range(num_batches):
        #print(str(epoche) + " Batch: " + str(batch))
        
        # input
        X_batch = X[batch_size*batch:(batch_size*batch+batch_size),:]
        
        # labels
        y_batch = y_vector[batch_size*batch:(batch_size*batch+batch_size)]
        
        # forward propagation
        l0 = X_batch
        l1 = softmax_batch(np.dot(l0,weights) + bias)
        
        # Mean Square Error
        mse = ((y_batch-l1)**2).mean()
        # print("Epoche: " + str(epoche)+ "; Batch: " + str(batch) +"; MSE = " + str(mse)) # <--------- MSE for every batch
        
        # The Error Weighted Derivative
        # using chain rule
        derivative_w = np.dot(l0.T, np.dot(2*(y_batch-l1),softmax_batch(l1, True)))
        derivative_b = np.dot(2*(y_batch-l1),softmax_batch(l1, True))
        
        # update weights (We add here because we did not use minus sign in the above calculation of derivative_w)
        # mse as learning rate
        weights += mse * derivative_w
        bias += mse * derivative_b.mean(axis=0)
        
    # accuracy with test data (validation)
    # if accuracy is better save weights and bias
    correct = 0
    result_test = np.argmax(softmax_batch(np.dot(X_test,weights) + bias), axis=1)

    # similarity is 0 if classification is correct
    # correct the number of 0s in similarity
    similarity = y_test - result_test
    correct = np.count_nonzero(similarity==0)

    # if better accuracy -> refresh
    if accuracy < (correct/len(similarity)):
        accuracy = correct/len(similarity)
        # save new optimal weights
        opt_weights = weights
        opt_bias = bias
        print("Epoch " + str(epoche) +"; accuracy: " + str(accuracy) + " (NEW)")
        no_refresh_counter = 0
    else:
        print("Epoch " + str(epoche) +"; accuracy: " + str(accuracy) )
        no_refresh_counter += 1

    # stop when validation error starts to stagnate
    if no_refresh_counter > 10:
      break
        

Epoch 0; accuracy: 0.8956 (NEW)
Epoch 1; accuracy: 0.905 (NEW)
Epoch 2; accuracy: 0.9103 (NEW)
Epoch 3; accuracy: 0.9118 (NEW)
Epoch 4; accuracy: 0.9136 (NEW)
Epoch 5; accuracy: 0.9146 (NEW)
Epoch 6; accuracy: 0.9163 (NEW)
Epoch 7; accuracy: 0.9163
Epoch 8; accuracy: 0.9172 (NEW)
Epoch 9; accuracy: 0.9174 (NEW)
Epoch 10; accuracy: 0.9184 (NEW)
Epoch 11; accuracy: 0.9184
Epoch 12; accuracy: 0.9184
Epoch 13; accuracy: 0.92 (NEW)
Epoch 14; accuracy: 0.9202 (NEW)
Epoch 15; accuracy: 0.9202
Epoch 16; accuracy: 0.9205 (NEW)
Epoch 17; accuracy: 0.9212 (NEW)
Epoch 18; accuracy: 0.9219 (NEW)
Epoch 19; accuracy: 0.9219
Epoch 20; accuracy: 0.9219
Epoch 21; accuracy: 0.9219
Epoch 22; accuracy: 0.9235 (NEW)
Epoch 23; accuracy: 0.9235
Epoch 24; accuracy: 0.9235
Epoch 25; accuracy: 0.9235
Epoch 26; accuracy: 0.9235
Epoch 27; accuracy: 0.9235
Epoch 28; accuracy: 0.9239 (NEW)
Epoch 29; accuracy: 0.9239
Epoch 30; accuracy: 0.9239
Epoch 31; accuracy: 0.9239
Epoch 32; accuracy: 0.9239
Epoch 33; accuracy: 